In [1]:
from ultralytics import YOLO
import cv2
import os
import numpy as np


# ---------------- CONFIG ----------------
model_path = r"runs\detect\train\weights\best.pt"
image_dir  = r"test\images"
roi_dir    = r"roi"
debug_dir  = r"roi_debug"

conf_thresh = 0.05
scale = 3.0   # ROI expansion factor
device = 0
# ----------------------------------------

os.makedirs(roi_dir, exist_ok=True)
os.makedirs(debug_dir, exist_ok=True)

model = YOLO(model_path)

image_files = [f for f in os.listdir(image_dir)
               if f.lower().endswith((".png", ".jpg", ".jpeg"))]

for img_name in image_files:
    img_path = os.path.join(image_dir, img_name)
    img = cv2.imread(img_path)

    if img is None:
        print(f"[WARN] Could not read {img_name}")
        continue

    H, W = img.shape[:2]

    results = model(img, conf=conf_thresh, device=device)
    result = results[0]

    if result.boxes is None or len(result.boxes) == 0:
        print(f"[INFO] No detection in {img_name}")
        continue

    # pick highest confidence detection
    boxes = result.boxes.xyxy.cpu().numpy()
    confs = result.boxes.conf.cpu().numpy()
    best_idx = np.argmax(confs)

    x1, y1, x2, y2 = map(int, boxes[best_idx])

    # -------- ROI EXPANSION --------
    cx = (x1 + x2) // 2
    cy = (y1 + y2) // 2

    w = int((x2 - x1) * scale)
    h = int((y2 - y1) * scale)

    x1_p = max(0, cx - w)
    y1_p = max(0, cy - h)
    x2_p = min(W, cx + w)
    y2_p = min(H, cy + h)

    roi = img[y1_p:y2_p, x1_p:x2_p]

    if roi.size == 0:
        print(f"[WARN] Empty ROI for {img_name}")
        continue

    # -------- SAVE ROI --------
    roi_path = os.path.join(roi_dir, img_name)
    cv2.imwrite(roi_path, roi)

    # -------- DEBUG IMAGE --------
    debug_img = img.copy()
    cv2.rectangle(debug_img, (x1_p, y1_p), (x2_p, y2_p), (0, 255, 0), 2)
    cv2.imwrite(os.path.join(debug_dir, img_name), debug_img)

    print(f"[OK] Saved ROI for {img_name}")

print("ROI extraction complete.")



0: 640x640 1 Stenosis, 5.4ms
Speed: 3.0ms preprocess, 5.4ms inference, 82.9ms postprocess per image at shape (1, 3, 640, 640)
[OK] Saved ROI for 14_002_5_0025_bmp.rf.debdb2e5cdec339506b1e5944ca03feb.jpg

0: 640x640 1 Stenosis, 9.1ms
Speed: 3.0ms preprocess, 9.1ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)
[OK] Saved ROI for 14_002_5_0031_bmp.rf.6fdc5e5ef440551c87822aa654fd49c9.jpg

0: 640x640 1 Stenosis, 5.9ms
Speed: 3.0ms preprocess, 5.9ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)
[OK] Saved ROI for 14_002_5_0046_bmp.rf.09b7eba5c2375d09bd4b9a1d36b951f6.jpg

0: 640x640 1 Stenosis, 5.1ms
Speed: 3.0ms preprocess, 5.1ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)
[OK] Saved ROI for 14_002_5_0048_bmp.rf.44733f7e6573d581ebaf90b7b3df3617.jpg

0: 640x640 1 Stenosis, 5.0ms
Speed: 3.4ms preprocess, 5.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)
[OK] Saved ROI for 14_002_8_0012_bmp.rf.976fcd7185b745133fc0b